In [ ]:
from PIL import Image
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('./gdrive', force_remount=True)

In [ ]:
# './gdrive/My Drive/Images/pickles/ecb128.pickle' 처럼 ecb, cbc, ofb 128 * 128 픽셀 이미지 array로 저장
# ECB는 육안으로 구분 가능
# CBC, OFB는 육안으로 구분 불가능
# 256 * 256 데이터셋보다 128 * 128 이 현재 모델에서 학습이 더 잘 됨
# 1채널 암호화와 3채널 암호화 육안으로 구분하기에는 차이 없지만 pickle에는 3채널 암호화 값 저장

In [ ]:
# pickle 파일 불러오기
import pickle
with open('./gdrive/My Drive/Images/pickles/ecb128.pickle', 'rb') as f:
  ecb = pickle.load(f)
  f.close()
with open('./gdrive/My Drive/Images/pickles/cbc128.pickle', 'rb') as f:
  cbc = pickle.load(f)
  f.close()
with open('./gdrive/My Drive/Images/pickles/ofb128.pickle', 'rb') as f:
  ofb = pickle.load(f)
  f.close()

In [ ]:
# ECB, CBC 구분 모델 

In [ ]:
# 정답 레이블 만들기
# 0 : ECB, 1 : CBC
ecb_label = [0 for _ in range(1000)]
cbc_label = [1 for _ in range(1000)]
label = ecb_label + cbc_label
label = np.array(label)

In [ ]:
# ECB, CBC 데이터 합친 뒤 데이터 섞기
data = np.concatenate((ecb, cbc), axis=0)
print(data.shape)

In [ ]:
import random

tmp = [[x, y] for x, y in zip(data, label)]
random.shuffle(tmp)
X = [n[0] for n in tmp]
Y = [n[1] for n in tmp]
X = np.array(X)
Y = np.array(Y)
print(X.shape)
print(Y.shape)

In [ ]:
# 2000개 데이터 중 200개 검증용, 1800개 훈련용, 255로 나눠서 이미지화
x_valid, x_train = X[:200] / 255, X[200:] / 255
y_valid, y_train = Y[:200], Y[200:]

In [ ]:
# 개, 고양이 구분 예제와 비슷하게
model = keras.models.Sequential([
              keras.layers.Conv2D(32, 3, activation="relu", padding="same", input_shape=[128,128,3]),
              keras.layers.MaxPooling2D(2, padding="valid"),
              keras.layers.Dropout(0.25),
              keras.layers.Conv2D(64, 3, activation="relu", padding="same"),
              keras.layers.MaxPooling2D(2, padding="valid"),
              keras.layers.Dropout(0.25),
              keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
              keras.layers.MaxPooling2D(2, padding="valid"),
              keras.layers.Dropout(0.25),
              keras.layers.Flatten(),
              keras.layers.Dense(512, activation="relu"),
              keras.layers.Dropout(0.5),
              keras.layers.Dense(2, activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)

In [ ]:
history = model.fit(x_train, y_train, epochs=30, validation_data=(x_valid, y_valid))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(acc, label="accurcy")
plt.plot(val_acc, label="val_accuracy")
plt.plot(loss, label="loss")
plt.plot(val_loss, label="val_loss")
plt.ylim(0,1.0)
plt.legend()
plt.show()

In [ ]:
# ECB, CBC 구분 - 최종 정확도 : 98%

In [ ]:
# CBC, OFB 구분 모델
data2 = np.concatenate((ofb, cbc), axis=0)
print(data2.shape)

In [ ]:
# 정답 레이블 만들기
# 0 : OFB, 1 : CBC
ofb_label = [0 for _ in range(1000)]
cbc_label = [1 for _ in range(1000)]
label2 = ofb_label + cbc_label
label2 = np.array(label2)

In [ ]:
# 데이터 섞기
import random

tmp2 = [[x, y] for x, y in zip(data2, label2)]
random.shuffle(tmp2)
X2 = [n[0] for n in tmp2]
Y2 = [n[1] for n in tmp2]
X2 = np.array(X2)
Y2 = np.array(Y2)
print(X2.shape)
print(Y2.shape)

In [ ]:
# 2000개 데이터 중 200개 검증용, 1800개 훈련용, 255로 나눠서 이미지화
x_valid2, x_train2 = X2[:200] / 255, X2[200:] / 255
y_valid2, y_train2 = Y2[:200], Y2[200:]

In [ ]:
# 위 모델과 마찬가지로 하기
model2 = model

In [ ]:
model2.summary()

In [ ]:
model2.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history2 = model2.fit(x_train2, y_train2, epochs=30, validation_data=(x_valid2, y_valid2))

In [ ]:
# 예상대로 CBC와 OFB는 구별 불가 (같은 모델 적용 시 50% 정확도)

In [ ]:
acc2 = history2.history['accuracy']
val_acc2 = history2.history['val_accuracy']
loss2 = history2.history['loss']
val_loss2 = history2.history['val_loss']
plt.plot(acc2, label="accurcy")
plt.plot(val_acc2, label="val_accuracy")
plt.plot(loss2, label="loss")
plt.plot(val_loss2, label="val_loss")
plt.ylim(0,1.0)
plt.legend()
plt.show()